In [62]:
import os
import re
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from models import (StreetType, CityType, Address, BelpostAddress,
                    get_database_engine, get_database_url)
from parser import setup_driver, search_postal_code
from rapidfuzz import fuzz

In [2]:
path = os.path.join("private_data", "ko_adresses.xlsx")
data = pd.read_excel(path)
data.head()

,to.street,to.building,soato.imns,soato.oblast,soato.district,soato.sovet,soato.tip,soato.name
0,УЛ. КОЛЬЦЕВАЯ,3,621,Минская,NaN,NaN,г.,Солигорск
1,УЛ. ВЛАДИМИРА ЛЕНИНА,238-3А,313,Витебская,NaN,NaN,г.,Орша
2,УЛ. ИНОГОРОДНЯЯ 3-Я,35,400,NaN,NaN,NaN,г.,Гомель
3,УЛ. БОРИСОВСКАЯ,2Г,603,Минская,Борисовский,Гливинский,д.,Гора
4,"УЛ. СУДОСТРОИТЕЛЬНАЯ, 10, пом. 316",NaN,419,Гомельская,Речицкий,Озерщинский,д.,Озерщина


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264997 entries, 0 to 264996
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   to.street       260015 non-null  object
 1   to.building     228084 non-null  object
 2   soato.imns      264997 non-null  int64 
 3   soato.oblast    134385 non-null  object
 4   soato.district  43071 non-null   object
 5   soato.sovet     39320 non-null   object
 6   soato.tip       264997 non-null  object
 7   soato.name      264997 non-null  object
dtypes: int64(1), object(7)
memory usage: 16.2+ MB


In [4]:
data["soato.tip"].value_counts()

soato.tip
г.     221616
аг.     13737
д.      13087
гп       6520
с/с      6246
р-н      1687
п.       1620
рп        246
кп        212
х.         24
пгт         2
Name: count, dtype: int64

In [10]:
[(t.name, t.value) for t in CityType]

[('AGROTOWN', 'АГРОГОРОДОК'),
 ('CITY', 'ГОРОД'),
 ('VILLAGE', 'ДЕРЕВНЯ'),
 ('SETTLEMENT', 'ПОСЕЛОК'),
 ('URBAN_SETTLEMENT', 'ГОРОДСКОЙ ПОСЕЛОК'),
 ('RESORT_SETTLEMENT', 'КУРОРТНЫЙ ПОСЕЛОК'),
 ('FARM', 'ХУТОР'),
 ('WORKERS_SETTLEMENT', 'РАБОЧИЙ ПОСЕЛОК'),
 ('SELO', 'СЕЛО'),
 ('RURAL_COUNCIL', 'СЕЛЬСОВЕТ'),
 ('SPECIAL_ECONOMIC_ZONE', 'ОСОБАЯ ЭКОНОМИЧЕСКАЯ ЗОНА')]

In [11]:
replace_dict = {
    "г.": "город",
    "аг.": "агрогородок",
    "гп": "городской поселок",
    "д. ": "деревня",
    "с/с": "сельский совет",
    "р-н": "район",
    "п.": "поселок",
    "рп": "рабочий поселок",
    "кп": "курортный поселок",
    "х.": "хутор",
    "пгт": "поселок городского типа",
}

In [89]:
engine = get_database_engine(echo=False)
session = Session(engine)
driver = setup_driver()

In [16]:
def build_address(
    region: str = None,
    district: str = None,
    sovet: str = None,
    city_type: str = None,
    city_name: str = None,
    street: str = None,
    building: str = None,
) -> str:
    """
    Универсальный конструктор адреса.
    Пропускает отсутствующие поля, не добавляя лишних слов.
    """

    parts = []

    if region:
        parts.append(f"{region} область")

    if district:
        parts.append(f"{district} район")

    if sovet:
        parts.append(f"{sovet} сельсовет")

    if city_name:  # тип города добавляем только если есть название
        if city_type:
            parts.append(f"{city_type} {city_name}")
        else:
            parts.append(city_name)

    if street:
        parts.append(street)

    if building:
        parts.append(building)

    return ", ".join(parts)

In [90]:
with session.begin():
    addr = session.query(Address).where(Address.id == 777).first()

    street = addr.street
    building = addr.building
    region = addr.soato_oblast
    district = addr.soato_district
    sovet = addr.soato_sovet
    city_name = addr.soato_name
    city_type = replace_dict.get(addr.soato_tip, "")

    address_str = build_address(
        region=region,
        district=district,
        sovet=sovet,
        city_type=city_type,
        city_name=city_name,
        street=street,
        building=building,
    )

    print("Составленный адрес: ", address_str)

    res = search_postal_code(driver, address_str)
    res = pd.DataFrame(res, columns=["Почтовый код","Область","Район",
                                     "Город","Улица","Номер дома"])
    session.close()
res

Составленный адрес:  город Гродно, ШОССЕ ИНДУРСКОЕ, 30
Открываем URL: https://www.belpost.by/Uznatpochtovyykod28indek?search=%D0%B3%D0%BE%D1%80%D0%BE%D0%B4%20%D0%93%D1%80%D0%BE%D0%B4%D0%BD%D0%BE%2C%20%D0%A8%D0%9E%D0%A1%D0%A1%D0%95%20%D0%98%D0%9D%D0%94%D0%A3%D0%A0%D0%A1%D0%9A%D0%9E%D0%95%2C%2030
Ожидание результатов поиска...


,Почтовый код,Область,Район,Город,Улица,Номер дома
0,230026,Гродненская,Гродненский,город Гродно,шоссе Индурское,"(7-17), 11/2, 15/1, 17/2, 17А"
1,230020,Гродненская,Гродненский,город Гродно,шоссе Индурское,"4/1, 4/2, 12, 14, 2, 4, 4А, 6, 8, 10"
2,230028,Гродненская,Гродненский,город Гродно,шоссе Индурское,"20, 30"
3,230003,Гродненская,Гродненский,город Гродно,шоссе Озерское,"26а, 18, 20, 20А, 20Б, 20В, 20Г, 20Д, 20/1, 22..."
4,230003,Гродненская,Гродненский,город Гродно,шоссе Скидельское,ВСЕ
5,211162,Витебская,Чашникский,город Новолукомль,переулок 30 Центральный,ВСЕ
6,246025,Гомельская,Гомельский,город Гомель,проезд 30 Лет Октября,"1, (2-6)"
7,231593,Гродненская,Мостовский,город Мосты,улица 30 Лет Победы,ВСЕ
8,222520,Минская,Борисовский,город Борисов,улица 30 Лет Влксм,"(25-29), 29А, 31, 34, (37-47), 18, 18/1, 18/2,..."
9,246025,Гомельская,Гомельский,город Гомель,улица 30 Лет Октября,"5, 8-13, 11А, 12А"


In [59]:
# --- Этап 1: фильтрация ---
def filter_addresses(df: pd.DataFrame, region: str, district: str, city: str) -> pd.DataFrame:
    """Жёсткая фильтрация DataFrame по региону, району, городу и диапазону домов."""
    mask = (
        (df["Область"].astype(str).str.contains(region, case=False, na=False) if region else True) &
        (df["Район"].astype(str).str.contains(district, case=False, na=False) if district else True) &
        (df["Город"].astype(str).str.contains(city, case=False, na=False) if city else True)
    )
    return df[mask]

In [91]:
first_filtered = filter_addresses(res,
                region=addr.soato_oblast,
                district=addr.soato_district,
                city=addr.soato_name)
first_filtered

,Почтовый код,Область,Район,Город,Улица,Номер дома
0,230026,Гродненская,Гродненский,город Гродно,шоссе Индурское,"(7-17), 11/2, 15/1, 17/2, 17А"
1,230020,Гродненская,Гродненский,город Гродно,шоссе Индурское,"4/1, 4/2, 12, 14, 2, 4, 4А, 6, 8, 10"
2,230028,Гродненская,Гродненский,город Гродно,шоссе Индурское,"20, 30"
3,230003,Гродненская,Гродненский,город Гродно,шоссе Озерское,"26а, 18, 20, 20А, 20Б, 20В, 20Г, 20Д, 20/1, 22..."
4,230003,Гродненская,Гродненский,город Гродно,шоссе Скидельское,ВСЕ


In [92]:
def add_similarity_scores(df: pd.DataFrame, target_string: str, column_name: str, scorer=fuzz.ratio):
    """
    Быстрое вычисление схожести с использованием rapidfuzz
    """
    df = df.copy()
    
    # Вычисляем схожесть для всех строк сразу
    scores = [scorer(str(x).lower(), str(target_string).lower()) for x in df[column_name]]
    
    df['similarity_score'] = scores
    df.sort_values(by="similarity_score", ascending=False, inplace=True)
    return df

In [93]:
add_similarity_scores(first_filtered,
                      target_string=addr.street,
                      column_name="Улица")

,Почтовый код,Область,Район,Город,Улица,Номер дома,similarity_score
0,230026,Гродненская,Гродненский,город Гродно,шоссе Индурское,"(7-17), 11/2, 15/1, 17/2, 17А",100.000000
1,230020,Гродненская,Гродненский,город Гродно,шоссе Индурское,"4/1, 4/2, 12, 14, 2, 4, 4А, 6, 8, 10",100.000000
2,230028,Гродненская,Гродненский,город Гродно,шоссе Индурское,"20, 30",100.000000
3,230003,Гродненская,Гродненский,город Гродно,шоссе Озерское,"26а, 18, 20, 20А, 20Б, 20В, 20Г, 20Д, 20/1, 22...",75.862069
4,230003,Гродненская,Гродненский,город Гродно,шоссе Скидельское,ВСЕ,75.000000
